In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc matplotlib numpy qiskit-addon-mpf qiskit-addon-utils rustworkx scipy

*Utilizzo stimato della QPU: Quattro minuti su un processore Heron r2 (NOTA: Questa è solo una stima. Il tempo di esecuzione potrebbe variare.)*

## Contesto

Questo tutorial dimostra come utilizzare una Formula Multi-Prodotto (MPF) per ottenere un errore di Trotter inferiore sull'osservabile rispetto a quello causato dal circuito di Trotter più profondo che effettivamente eseguiremo.
Le MPF riducono l'errore di Trotter della dinamica hamiltoniana attraverso una combinazione pesata di diverse esecuzioni di circuiti. Considerate il compito di trovare i valori di aspettazione di osservabili per lo stato quantistico
$\rho(t)=e^{-i H t} \rho(0) e^{i H t}$ con l'hamiltoniana $H$. Si possono utilizzare le Formule di Prodotto (PF) per approssimare l'evoluzione temporale $e^{-i H t}$ procedendo come segue:

- Scrivere l'hamiltoniana $H$ come $H=\sum_{a=1}^d F_a,$ dove $F_a$ sono operatori hermitiani tali che ciascun unitario corrispondente possa essere implementato in modo efficiente su un dispositivo quantistico.
- Approssimare i termini $F_a$ che non commutano tra loro.

Quindi, la PF del primo ordine (formula di Lie-Trotter) è:

$$S_1(t):=\prod_{a=1}^d e^{-i F_a t},$$

che ha un termine di errore quadratico $S_1(t)=e^{-i H t}+\mathcal{O}\left(t^{2}\right)$. Si possono anche utilizzare PF di ordine superiore (formule di Lie-Trotter-Suzuki), che convergono più rapidamente e sono definite ricorsivamente come:

$$S_2(t):=\prod_{a=1}^d e^{-i F_a t/2}\prod_{a=1}^d e^{-i F_a t/2}$$

$$S_{2 \chi}(t):= S_{2 \chi -2}(s_{\chi}t)^2 S_{2 \chi -2}((1-4s_{\chi})t)S_{2 \chi -2}(s_{\chi}t)^2,$$

dove $\chi$ è l'ordine della PF simmetrica e $s_p = \left( 4 - 4^{1/(2p-1)} \right)^{-1}$. Per evoluzioni temporali lunghe, si può dividere l'intervallo di tempo $t$ in $k$ intervalli, chiamati passi di Trotter, di durata $t/k$ e approssimare l'evoluzione temporale in ciascun intervallo con una formula di prodotto di ordine $\chi$ $S_{\chi}$. Quindi, la PF di ordine $\chi$ per l'operatore di evoluzione temporale su $k$ passi di Trotter è:

$$ S_{\chi}^{k}(t) = \left[ S_{\chi} \left( \frac{t}{k} \right)\right]^k = e^{-i H t}+O\left(t \left( \frac{t}{k} \right)^{\chi} \right)$$

dove il termine di errore diminuisce con il numero di passi di Trotter $k$ e l'ordine $\chi$ della PF.

Dato un intero $k \geq 1$ e una formula di prodotto $S_{\chi}(t)$, lo stato approssimato evoluto nel tempo $\rho_k(t)$ può essere ottenuto da $\rho_0$ applicando $k$ iterazioni della formula di prodotto $S_{\chi}\left(\frac{t}{k}\right)$.

$$
\rho_k(t)=S_{\chi}\left(\frac{t}{k}\right)^k \rho_0 S_{\chi}\left(\frac{t}{k}\right)^{-k}
$$

$\rho_k(t)$ è un'approssimazione di $\rho(t)$ con l'errore di approssimazione di Trotter ||$\rho_k(t)-\rho(t) ||$. Se consideriamo una combinazione lineare di approssimazioni di Trotter di $\rho(t)$:

$$
\mu(t) = \sum_{j}^{l} x_j \rho^{k_j}_{j}\left(\frac{t}{k_j}\right) + \text{un certo errore di Trotter residuo} \, ,
$$

dove $x_j$ sono i nostri coefficienti di ponderazione, $\rho^{k_j}_j$ è la matrice densità corrispondente allo stato puro ottenuto evolvendo lo stato iniziale con la formula di prodotto, $S^{k_j}_{\chi}$, che coinvolge $k_j$ passi di Trotter, e $j \in {1, ..., l}$ indicizza il numero di PF che compongono la MPF. Tutti i termini in $\mu(t)$ utilizzano la stessa formula di prodotto $S_{\chi}(t)$ come base.
L'obiettivo è migliorare ||$\rho_k(t)-\rho(t) \|$ trovando $\mu(t)$ con $\|\mu(t)-\rho(t)\|$ ancora più basso.

* $\mu(t)$ non deve necessariamente essere uno stato fisico poiché $x_i$ non deve essere positivo. L'obiettivo qui è minimizzare l'errore nel valore di aspettazione degli osservabili e non trovare una sostituzione fisica per $\rho(t)$.
* $k_j$ determina sia la profondità del circuito che il livello di approssimazione di Trotter. Valori più piccoli di $k_j$ portano a circuiti più brevi, che incorrono in minori errori di circuito ma saranno un'approssimazione meno accurata dello stato desiderato.

La chiave qui è che l'errore di Trotter residuo dato da $\mu(t)$ è più piccolo dell'errore di Trotter che si otterrebbe semplicemente utilizzando il valore più grande di $k_j$.

Potete vedere l'utilità di questo da due prospettive:

1. Per un budget fisso di passi di Trotter che potete eseguire, potete ottenere risultati con un errore di Trotter che è complessivamente più piccolo.
2. Dato un numero target di passi di Trotter che è troppo grande da eseguire, potete utilizzare la MPF per trovare una collezione di circuiti a profondità inferiore da eseguire che risulta in un errore di Trotter simile.

## Requisiti

Prima di iniziare questo tutorial, assicuratevi di avere installato quanto segue:

* Qiskit SDK v1.0 o successivo, con supporto per la [visualizzazione](https://docs.quantum.ibm.com/api/qiskit/visualization)
* Qiskit Runtime v0.22 o successivo (`pip install qiskit-ibm-runtime`)
* MPF Qiskit addons (`pip install qiskit_addon_mpf`)
* Qiskit addons utils (`pip install qiskit_addon_utils`)
* Libreria Quimb (`pip install quimb`)
* Libreria Qiskit Quimb (`pip install qiskit-quimb`)
* Numpy v0.21 per la compatibilità tra i pacchetti (`pip install numpy==0.21`)

## Parte I. Esempio su piccola scala

### Esplorare la stabilità della MPF

Non c'è una restrizione ovvia sulla scelta del numero di passi di Trotter $k_j$ che compongono lo stato MPF $\mu(t)$. Tuttavia, questi devono essere scelti con attenzione per evitare instabilità nei valori di aspettazione risultanti calcolati da $\mu(t)$. Una buona regola generale è impostare il passo di Trotter più piccolo $k_{\text{min}}$ in modo che $t/k_{\text{min}} \lt 1$. Se volete saperne di più su questo e su come scegliere gli altri valori $k_j$, fate riferimento alla guida [Come scegliere i passi di Trotter per una MPF](https://qiskit.github.io/qiskit-addon-mpf/how_tos/choose_trotter_steps.html).

Nell'esempio seguente esploriamo la stabilità della soluzione MPF calcolando il valore di aspettazione della magnetizzazione per un intervallo di tempi utilizzando diversi stati evoluti nel tempo. Nello specifico, confrontiamo i valori di aspettazione calcolati da ciascuna delle evoluzioni temporali approssimate implementate con i corrispondenti passi di Trotter e i vari modelli MPF (coefficienti statici e dinamici) con i valori esatti dell'osservabile evoluto nel tempo. Innanzitutto, definiamo i parametri per le formule di Trotter e i tempi di evoluzione

In [1]:
import numpy as np

mpf_trotter_steps = [1, 2, 4]
order = 2
symmetric = False

trotter_times = np.arange(0.5, 1.55, 0.1)
exact_evolution_times = np.arange(trotter_times[0], 1.55, 0.05)

Per questo esempio utilizzeremo lo stato di Neel come stato iniziale $\vert \text{Neel} \rangle = \vert 0101...01 \rangle$ e il modello di Heisenberg su una linea di 10 siti per l'hamiltoniana che governa l'evoluzione temporale

$$
\hat{\mathcal{H}}_{Heis} = J \sum_{i=1}^{L-1} \left(X_i X_{(i+1)}+Y_i Y_{(i+1)}+ Z_i Z_{(i+1)} \right) \, ,
$$

dove $J$ è l'intensità dell'accoppiamento per i bordi tra i vicini più prossimi.

In [2]:
from qiskit.transpiler import CouplingMap
from rustworkx.visualization import graphviz_draw
from qiskit_addon_utils.problem_generators import generate_xyz_hamiltonian
import numpy as np


L = 10

# Generate some coupling map to use for this example
coupling_map = CouplingMap.from_line(L, bidirectional=False)
graphviz_draw(coupling_map.graph, method="circo")

# Get a qubit operator describing the Heisenberg field model
hamiltonian = generate_xyz_hamiltonian(
    coupling_map,
    coupling_constants=(1.0, 1.0, 1.0),
    ext_magnetic_field=(0.0, 0.0, 0.0),
)


print(hamiltonian)

SparsePauliOp(['IIIIIIIXXI', 'IIIIIIIYYI', 'IIIIIIIZZI', 'IIIIIXXIII', 'IIIIIYYIII', 'IIIIIZZIII', 'IIIXXIIIII', 'IIIYYIIIII', 'IIIZZIIIII', 'IXXIIIIIII', 'IYYIIIIIII', 'IZZIIIIIII', 'IIIIIIIIXX', 'IIIIIIIIYY', 'IIIIIIIIZZ', 'IIIIIIXXII', 'IIIIIIYYII', 'IIIIIIZZII', 'IIIIXXIIII', 'IIIIYYIIII', 'IIIIZZIIII', 'IIXXIIIIII', 'IIYYIIIIII', 'IIZZIIIIII', 'XXIIIIIIII', 'YYIIIIIIII', 'ZZIIIIIIII'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j])


The observable that we will be measuring is magnetization on a pair of qubits in the middle of the chain.

In [ ]:
from qiskit.quantum_info import SparsePauliOp

observable = SparsePauliOp.from_sparse_list(
    [("ZZ", (L // 2 - 1, L // 2), 1.0)], num_qubits=L
)
print(observable)

SparsePauliOp(['IIIIZZIIII'],
              coeffs=[1.+0.j])


L'osservabile che misureremo è la magnetizzazione su una coppia di qubit al centro della catena.

In [4]:
from qiskit.circuit.library import XXPlusYYGate
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes.optimization.collect_and_collapse import (
    CollectAndCollapse,
    collect_using_filter_function,
    collapse_to_operation,
)
from functools import partial


def filter_function(node):
    return node.op.name in {"rxx", "ryy"}


collect_function = partial(
    collect_using_filter_function,
    filter_function=filter_function,
    split_blocks=True,
    min_block_size=1,
)


def collapse_to_xx_plus_yy(block):
    param = 0.0
    for node in block.data:
        param += node.operation.params[0]
    return XXPlusYYGate(param)


collapse_function = partial(
    collapse_to_operation,
    collapse_function=collapse_to_xx_plus_yy,
)

pm = PassManager()
pm.append(CollectAndCollapse(collect_function, collapse_function))

Then we create the circuits implementing the approximate Trotter time-evolutions.

In [5]:
from qiskit.synthesis import SuzukiTrotter
from qiskit_addon_utils.problem_generators import (
    generate_time_evolution_circuit,
)
from qiskit import QuantumCircuit


# Initial Neel state preparation
initial_state_circ = QuantumCircuit(L)
initial_state_circ.x([i for i in range(L) if i % 2 != 0])


all_circs = []
for total_time in trotter_times:
    mpf_trotter_circs = [
        generate_time_evolution_circuit(
            hamiltonian,
            time=total_time,
            synthesis=SuzukiTrotter(reps=num_steps, order=order),
        )
        for num_steps in mpf_trotter_steps
    ]

    mpf_trotter_circs = pm.run(
        mpf_trotter_circs
    )  # Collect XX and YY into XX + YY

    mpf_circuits = [
        initial_state_circ.compose(circuit) for circuit in mpf_trotter_circs
    ]
    all_circs.append(mpf_circuits)

In [6]:
mpf_circuits[-1].draw("mpl", fold=-1)

<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/92dc20a7-0.avif" alt="Output of the previous code cell" />

Quindi creiamo i circuiti che implementano le evoluzioni temporali approssimate di Trotter.

In [24]:
from copy import deepcopy
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager


aer_sim = AerSimulator()
estimator = Estimator(mode=aer_sim)

mpf_expvals_all_times, mpf_stds_all_times = [], []
for t, mpf_circuits in zip(trotter_times, all_circs):
    mpf_expvals = []
    circuits = [deepcopy(circuit) for circuit in mpf_circuits]
    pm_sim = generate_preset_pass_manager(
        backend=aer_sim, optimization_level=3
    )
    isa_circuits = pm_sim.run(circuits)
    result = estimator.run(
        [(circuit, observable) for circuit in isa_circuits], precision=0.005
    ).result()
    mpf_expvals = [res.data.evs for res in result]
    mpf_stds = [res.data.stds for res in result]
    mpf_expvals_all_times.append(mpf_expvals)
    mpf_stds_all_times.append(mpf_stds)

We also calculate the exact expectation values for comparison.

In [ ]:
from scipy.linalg import expm
from qiskit.quantum_info import Statevector

exact_expvals = []
for t in exact_evolution_times:
    # Exact expectation values
    exp_H = expm(-1j * t * hamiltonian.to_matrix())
    initial_state = Statevector(initial_state_circ).data
    time_evolved_state = exp_H @ initial_state

    exact_obs = (
        time_evolved_state.conj()
        @ observable.to_matrix()
        @ time_evolved_state
    ).real
    exact_expvals.append(exact_obs)

![Output of the previous code cell](../docs/images/tutorials/multi-product-formula/extracted-outputs/92dc20a7-0.avif)

Successivamente, calcoliamo i valori di aspettazione evoluti nel tempo dai circuiti di Trotter.

In [9]:
from qiskit_addon_mpf.static import setup_static_lse

lse = setup_static_lse(mpf_trotter_steps, order=order, symmetric=symmetric)

Calcoliamo anche i valori di aspettazione esatti per il confronto.

In [10]:
lse.A

array([[1.      , 1.      , 1.      ],
       [1.      , 0.25    , 0.0625  ],
       [1.      , 0.125   , 0.015625]])

#### Coefficienti MPF statici
Le MPF statiche sono quelle in cui i valori $x_j$ non dipendono dal tempo di evoluzione, $t$. Consideriamo la PF di ordine $\chi = 1$ con $k_j$ passi di Trotter, questa può essere scritta come:

$$ S_1^{k_j}\left( \frac{t}{k_j} \right)=e^{-i H t}+ \sum_{n=1}^{\infty} A_n \frac{t^{n+1}}{k_j^n}  $$

dove $A_n$ sono matrici che dipendono dai commutatori dei termini $F_a$ nella decomposizione dell'hamiltoniana. È importante notare che $A_n$ stessi sono indipendenti dal tempo e dal numero di passi di Trotter $k_j$. Pertanto, è possibile cancellare i termini di errore di ordine inferiore che contribuiscono a $\mu(t)$ con una scelta attenta dei pesi $x_j$ della combinazione lineare. Per cancellare l'errore di Trotter per i primi $l-1$ termini (questi daranno i contributi maggiori poiché corrispondono al numero inferiore di passi di Trotter) nell'espressione per $\mu(t)$, i coefficienti $x_j$ devono soddisfare le seguenti equazioni:

$$ \sum_{j=1}^l x_j = 1 $$
$$ \sum_{j=1}^{l-1} \frac{x_j}{k_j^{n}} = 0 $$

con $n=0, ... l-2$. La prima equazione garantisce che non ci sia bias nello stato costruito $\mu(t)$, mentre la seconda equazione assicura la cancellazione degli errori di Trotter. Per PF di ordine superiore, la seconda equazione diventa $ \sum_{j=1}^{l-1} \frac{x_j}{k_j^{\eta}} = 0 $ dove $\eta = \chi + 2n$ per PF simmetriche e $\eta = \chi + n$ altrimenti, con $n=0, ..., l-2$. L'errore risultante (Rif. [\[1\]](#references),[\[2\]](#references)) è quindi

$$ \epsilon = \mathcal{O} \left( \frac{t^{l+1}}{k_1^l} \right).$$

Determinare i coefficienti MPF statici per un dato insieme di valori $k_j$ equivale a risolvere il sistema lineare di equazioni definito dalle due equazioni precedenti per le variabili $x_j$: $Ax=b$. Dove $x$ sono i nostri coefficienti di interesse, $A$ è una matrice che dipende da $k_j$ e dal tipo di PF che utilizziamo ($S$), e $b$ è un vettore di vincoli. Specificamente:

$$A_{0,j} = 1 $$
$$A_{i>0,j} = k_{j}^{-(\chi + s(i-1))}$$
$$b_0 = 1$$
$$b_{i>0} = 0 $$

dove $\chi$ è l'``order``, $s$ è $2$ se ``symmetric`` è ``True`` e $1$ altrimenti, $k_{j}$ sono i ``trotter_steps``, e $x$ sono le variabili da risolvere. Gli indici $i$ e $j$ partono da $0$. Possiamo anche visualizzare questo in forma matriciale:

$$
A =
\begin{bmatrix}
A_{0,0} & A_{0,1} & A_{0,2} & ... \\
A_{1,0} & A_{1,1} & A_{1,2} & ...  \\
A_{2,0} & A_{2,1} & A_{2,2} & ...  \\
... & ... & ... & ...
\end{bmatrix} =
\begin{bmatrix}
1 & 1 & 1 & ... \\
k_{0}^{-(\chi + s(1-1))} & k_{1}^{-(\chi + s(1-1))} & k_{2}^{-(\chi + s(1-1))} & ... \\
k_{0}^{-(\chi + s(2-1))} & k_{1}^{-(\chi + s(2-1))} & k_{2}^{-(\chi + s(2-1))} & ... \\
... & ... & ... & ...
\end{bmatrix}
$$

e

$$
b =
\begin{bmatrix}
b_{0} \\
b_{1} \\
b_{2}  \\
...
\end{bmatrix} =
\begin{bmatrix}
1 \\
0 \\
0  \\
...
\end{bmatrix}
$$

Per maggiori dettagli, fate riferimento alla documentazione del Sistema Lineare di Equazioni ([LSE](https://qiskit.github.io/qiskit-addon-mpf/stubs/qiskit_addon_mpf.static.LSE.html)).

Possiamo trovare una soluzione per $x$ analiticamente come $x = A^{-1}b$; vedete ad esempio i Rif. [\[1\]](#references) o [\[2\]](#references).
Tuttavia, questa soluzione esatta può essere "mal condizionata", risultando in norme L1 molto grandi dei nostri coefficienti, $x$, che possono portare a cattive prestazioni della MPF.
Invece, si può anche ottenere una soluzione approssimata che minimizza la norma L1 di $x$ al fine di tentare di ottimizzare il comportamento della MPF.
##### Impostare l'LSE
Ora che abbiamo scelto i nostri valori $k_j$, dobbiamo prima costruire l'LSE, $Ax=b$ come spiegato sopra.
La matrice $A$ dipende non solo da $k_j$ ma anche dalla nostra scelta di PF, in particolare dal suo _ordine_.
Inoltre, potreste tenere conto se la PF è simmetrica o no (vedete [\[1\]](#references)) impostando `symmetric=True/False`.
Tuttavia, questo non è richiesto, come mostrato dal Rif. [\[2\]](#references).

In [11]:
lse.b

array([1., 0., 0.])

Esaminiamo i valori scelti sopra per costruire la matrice $A$ e il vettore $b$. Con $j=0,1, 2$ passi di Trotter $k_j = [1, 2, 4]$, ordine $\chi = 2$ e scelta di passi di Trotter non simmetrici ($s=1$), abbiamo che gli elementi della matrice $A$ sotto la prima riga sono determinati dall'espressione $A_{i>0,j} = k_{j}^{-(2 + 1(i-1))}$, specificamente:

$$ A_{0,0} = A_{0,1} = A_{0,2} =  1 $$
$$ A_{1,j} = k_{j}^{-1}  \rightarrow A_{1,0} = \frac{1}{1^2}, \;, A_{1,1} = \frac{1}{2^2}, \;, A_{1,2} = \frac{1}{4^2}$$
$$ A_{2,j} = k_{j}^{-2}  \rightarrow A_{2,0} = \frac{1}{1^3}, \;, A_{2,1} = \frac{1}{2^3}, \;, A_{2,2} = \frac{1}{4^3}$$

o in forma matriciale:

$$
A =
\begin{bmatrix}
1 & 1 & 1\\
1 & \frac{1}{2^2} & \frac{1}{4^2}  \\
1 & \frac{1}{2^3} & \frac{1}{4^3}  \\
\end{bmatrix}
$$

Questo è possibile vederlo ispezionando l'oggetto `lse`:

In [12]:
mpf_coeffs = lse.solve()
print(
    f"The static coefficients associated with the ansatze are: {mpf_coeffs}"
)

The static coefficients associated with the ansatze are: [ 0.04761905 -0.57142857  1.52380952]


##### Optimize for $x$ using an exact model

Alternatively to computing $x=A^{-1}b$, you can also use [setup_exact_model](https://qiskit.github.io/qiskit-addon-mpf/stubs/qiskit_addon_mpf.static.setup_exact_model.html) to construct a [cvxpy.Problem](https://www.cvxpy.org/api_reference/cvxpy.problems.html#cvxpy.Problem) instance that uses the LSE as constraints and whose optimal solution will yield $x$.

In the next section, it will be clear why this interface exists.

In [13]:
from qiskit_addon_mpf.costs import setup_exact_problem

model_exact, coeffs_exact = setup_exact_problem(lse)
model_exact.solve()
print(coeffs_exact.value)

[ 0.04761905 -0.57142857  1.52380952]


Mentre il vettore di vincoli $b$ ha i seguenti elementi:
$$ b_{0} = 1 $$
$$ b_1 = b_2 = 0 $$

Quindi,

$$
b =
\begin{bmatrix}
1 \\
0 \\
0
\end{bmatrix}
$$

E similmente in `lse`:

In [14]:
print(
    "L1 norm of the exact coefficients:",
    np.linalg.norm(coeffs_exact.value, ord=1),
)  # ord specifies the norm. ord=1 is for L1

L1 norm of the exact coefficients: 2.1428571428556378


##### Optimize for $x$ using an approximate model

It might happen that the L1 norm for the chosen set of $k_j$ values is deemed too high.
If that is the case and you cannot choose a different set of $k_j$ values, you can use an approximate solution to the LSE instead of an exact one.

To do so, simply use [setup_approximate_model](https://qiskit.github.io/qiskit-addon-mpf/stubs/qiskit_addon_mpf.static.setup_approximate_model.html) to construct a different [cvxpy.Problem](https://www.cvxpy.org/api_reference/cvxpy.problems.html#cvxpy.Problem) instance, which constrains the L1-norm to a chosen threshold while minimizing the difference of $Ax$ and $b$.

In [16]:
from qiskit_addon_mpf.costs import setup_sum_of_squares_problem

model_approx, coeffs_approx = setup_sum_of_squares_problem(
    lse, max_l1_norm=1.5
)
model_approx.solve()
print(coeffs_approx.value)
print(
    "L1 norm of the approximate coefficients:",
    np.linalg.norm(coeffs_approx.value, ord=1),
)

[-1.10294118e-03 -2.48897059e-01  1.25000000e+00]
L1 norm of the approximate coefficients: 1.5


L'oggetto `lse` ha metodi per trovare i coefficienti statici $x_j$ che soddisfano il sistema di equazioni.

In [17]:
from qiskit_addon_utils.slicing import slice_by_depth
from qiskit_addon_mpf.backends.tenpy_layers import LayerModel
from qiskit_addon_mpf.backends.tenpy_layers import LayerwiseEvolver
from functools import partial

# Create approximate time-evolution circuits
single_2nd_order_circ = generate_time_evolution_circuit(
    hamiltonian, time=1.0, synthesis=SuzukiTrotter(reps=1, order=order)
)
single_2nd_order_circ = pm.run(single_2nd_order_circ)  # collect XX and YY

# Find layers in the circuit
layers = slice_by_depth(single_2nd_order_circ, max_slice_depth=1)

# Create tensor network models
models = [
    LayerModel.from_quantum_circuit(layer, conserve="Sz") for layer in layers
]

# Create the time-evolution object
approx_factory = partial(
    LayerwiseEvolver,
    layers=models,
    options={
        "preserve_norm": False,
        "trunc_params": {
            "chi_max": 64,
            "svd_min": 1e-8,
            "trunc_cut": None,
        },
        "max_delta_t": 2,
    },
)

<Admonition type="warning">
The options of `LayerwiseEvolver` that determine the details of the tensor network simulation must be chosen carefully to avoid setting up an ill-defined optimization problem.
</Admonition>

We then set up the exact evolver (for example, [`ExactEvolverFactory`](https://qiskit.github.io/qiskit-addon-mpf/apidocs/qiskit_addon_mpf.dynamic.html#qiskit_addon_mpf.dynamic.ExactEvolverFactory)), which returns an [`Evolver`](https://qiskit.github.io/qiskit-addon-mpf/apidocs/qiskit_addon_mpf.backends.html#qiskit_addon_mpf.backends.Evolver) object computing the true or “reference” time-evolution. Realistically, we would approximate the exact evolution by using a higher-order Suzuki–Trotter formula or another reliable method with a small time step. Below, we approximate the exact time-evolved state with a fourth-order Suzuki-Trotter formula using a small time step `dt=0.1`, which means the number of Trotter steps used at time $t$ is $k=t/dt$. We also specify some TeNPy-specific truncation options to bound the maximum bond dimension of the underlying tensor network, as well as the minimum singular values of the split tensor network bonds. These parameters can affect the accuracy of the expectation value calculated with the dynamic MPF coefficients, so it is important to explore a range of values to find the optimal balance between computational time and accuracy. Note that the calculation of the MPF coefficients does not rely of the expectation value of the PF obtained from hardware execution, and therefore it can be tuned in post-processing.

In [20]:
single_4th_order_circ = generate_time_evolution_circuit(
    hamiltonian, time=1.0, synthesis=SuzukiTrotter(reps=1, order=4)
)
single_4th_order_circ = pm.run(single_4th_order_circ)
exact_model_layers = [
    LayerModel.from_quantum_circuit(layer, conserve="Sz")
    for layer in slice_by_depth(single_4th_order_circ, max_slice_depth=1)
]

exact_factory = partial(
    LayerwiseEvolver,
    layers=exact_model_layers,
    dt=0.1,
    options={
        "preserve_norm": False,
        "trunc_params": {
            "chi_max": 64,
            "svd_min": 1e-8,
            "trunc_cut": None,
        },
        "max_delta_t": 2,
    },
)

##### Ottimizzare per $x$ usando un modello esatto
Alternativamente al calcolo di $x=A^{-1}b$, potete anche utilizzare [setup_exact_model](https://qiskit.github.io/qiskit-addon-mpf/stubs/qiskit_addon_mpf.static.setup_exact_model.html) per costruire un'istanza di [cvxpy.Problem](https://www.cvxpy.org/api_reference/cvxpy.problems.html#cvxpy.Problem) che utilizza l'LSE come vincoli e la cui soluzione ottimale produrrà $x$.

Nella prossima sezione, sarà chiaro perché esiste questa interfaccia.

In [ ]:
from qiskit_addon_mpf.backends.tenpy_tebd import MPOState
from qiskit_addon_mpf.backends.tenpy_tebd import MPS_neel_state


def identity_factory():
    return MPOState.initialize_from_lattice(models[0].lat, conserve=True)


mps_initial_state = MPS_neel_state(models[0].lat)

For each time step $t$ we set up the dynamic linear system of equations with the [`setup_dynamic_lse`](https://qiskit.github.io/qiskit-addon-mpf/apidocs/qiskit_addon_mpf.dynamic.html) method. The corresponding object contains the information about the dynamic MPF problem: `lse.A` gives the Gram matrix $M$ while `lse.b` gives the overlap $L$. We can then solve the LSE (when not ill-defined) to find the dynamic coefficients using the `setup_frobenius_problem`. It is important to note the difference with the static coefficients, which only depend on the details of the product formula used and are independent of the details of the time-evolution (Hamiltonian and initial state).

In [22]:
from qiskit_addon_mpf.dynamic import setup_dynamic_lse
from qiskit_addon_mpf.costs import setup_frobenius_problem

mpf_dynamic_coeffs_list = []
for t in trotter_times:
    print(f"Computing dynamic coefficients for time={t}")
    lse = setup_dynamic_lse(
        mpf_trotter_steps,
        t,
        identity_factory,
        exact_factory,
        approx_factory,
        mps_initial_state,
    )
    problem, coeffs = setup_frobenius_problem(lse)
    try:
        problem.solve()
        mpf_dynamic_coeffs_list.append(coeffs.value)
    except Exception as error:
        mpf_dynamic_coeffs_list.append(np.zeros(len(mpf_trotter_steps)))
        print(error, "Calculation Failed for time", t)
    print("")

Computing dynamic coefficients for time=0.5

Computing dynamic coefficients for time=0.6

Computing dynamic coefficients for time=0.7

Computing dynamic coefficients for time=0.7999999999999999

Computing dynamic coefficients for time=0.8999999999999999

Computing dynamic coefficients for time=0.9999999999999999

Computing dynamic coefficients for time=1.0999999999999999

Computing dynamic coefficients for time=1.1999999999999997

Computing dynamic coefficients for time=1.2999999999999998

Computing dynamic coefficients for time=1.4

Computing dynamic coefficients for time=1.4999999999999998



Come indicatore se una MPF costruita con questi coefficienti produrrà buoni risultati, possiamo utilizzare la norma L1 (vedete anche Rif. [\[1\]](#references)).

In [ ]:
import matplotlib.pyplot as plt

sym = {1: "^", 2: "s", 4: "p"}
# Get expectation values at all times for each Trotter step
for k, step in enumerate(mpf_trotter_steps):
    trotter_curve, trotter_curve_error = [], []
    for trotter_expvals, trotter_stds in zip(
        mpf_expvals_all_times, mpf_stds_all_times
    ):
        trotter_curve.append(trotter_expvals[k])
        trotter_curve_error.append(trotter_stds[k])

    plt.errorbar(
        trotter_times,
        trotter_curve,
        yerr=trotter_curve_error,
        alpha=0.5,
        markersize=4,
        marker=sym[step],
        color="grey",
        label=f"{mpf_trotter_steps[k]} Trotter steps",
    )  # , , )

# Get expectation values at all times for the static MPF with exact coeffs
exact_mpf_curve, exact_mpf_curve_error = [], []
for trotter_expvals, trotter_stds in zip(
    mpf_expvals_all_times, mpf_stds_all_times
):
    mpf_std = np.sqrt(
        sum(
            [
                (coeff**2) * (std**2)
                for coeff, std in zip(coeffs_exact.value, trotter_stds)
            ]
        )
    )
    exact_mpf_curve_error.append(mpf_std)
    exact_mpf_curve.append(trotter_expvals @ coeffs_exact.value)

plt.errorbar(
    trotter_times,
    exact_mpf_curve,
    yerr=exact_mpf_curve_error,
    markersize=4,
    marker="o",
    label="Static MPF - Exact",
    color="purple",
)


# Get expectation values at all times for the static MPF with approximate
approx_mpf_curve, approx_mpf_curve_error = [], []
for trotter_expvals, trotter_stds in zip(
    mpf_expvals_all_times, mpf_stds_all_times
):
    mpf_std = np.sqrt(
        sum(
            [
                (coeff**2) * (std**2)
                for coeff, std in zip(coeffs_approx.value, trotter_stds)
            ]
        )
    )
    approx_mpf_curve_error.append(mpf_std)
    approx_mpf_curve.append(trotter_expvals @ coeffs_approx.value)

plt.errorbar(
    trotter_times,
    approx_mpf_curve,
    yerr=approx_mpf_curve_error,
    markersize=4,
    marker="o",
    color="orange",
    label="Static MPF - Approximate",
)

# # Get expectation values at all times for the dynamic MPF
dynamic_mpf_curve, dynamic_mpf_curve_error = [], []
for trotter_expvals, trotter_stds, dynamic_coeffs in zip(
    mpf_expvals_all_times, mpf_stds_all_times, mpf_dynamic_coeffs_list
):
    mpf_std = np.sqrt(
        sum(
            [
                (coeff**2) * (std**2)
                for coeff, std in zip(dynamic_coeffs, trotter_stds)
            ]
        )
    )
    dynamic_mpf_curve_error.append(mpf_std)
    dynamic_mpf_curve.append(trotter_expvals @ dynamic_coeffs)

plt.errorbar(
    trotter_times,
    dynamic_mpf_curve,
    yerr=dynamic_mpf_curve_error,
    markersize=4,
    marker="o",
    color="pink",
    label="Dynamic MPF",
)


plt.plot(
    exact_evolution_times,
    exact_expvals,
    lw=3,
    color="red",
    label="Exact time-evolution",
)


plt.title(
    f"Expectation values for (ZZ,{(L//2-1, L//2)}) as a function of time"
)
plt.xlabel("Time")
plt.ylabel("Expectation Value")
plt.legend()
plt.grid()

<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/2da9c948-0.avif" alt="Output of the previous code cell" />

In the cases like the example above, where the $k=1$ PF behaves poorly at all times, the quality of the dynamic MPF results is also heavily affected. In such situations, it is useful to investigate the possibility of using individual PFs with higher number of Trotter steps to improve the overall quality of the results. In these simulations, we see the interplay of different types of errors: error from finite sampling, and Trotter error from the product formulas. MPF helps to reduce the Trotter error due to the product formulas but incur in higher sampling error compared to the product formulas. This can be advantageous, as product formulas can reduce the sampling error with increased sampling, but the systematic error due to the Trotter approximation remains untouched.

Another interesting behavior that we can observe from the plot is that the expectation value for the PF for $k=1$ starts to behave erratically (on top of not being a good approximation for the exact one) at times for which $t/k > 1 $, as explained in the [guide](https://qiskit.github.io/qiskit-addon-mpf/how_tos/choose_trotter_steps.html) on how to choose the number of Trotter steps.

### Step 1: Map classical inputs to a quantum problem
Let's now consider a single time $t=1.0$ and calculate the expectation value of the magnetization with the various methods using one QPU. The particular choice of $t$ was done so to maximize the difference between the various methods and observe their relative efficacy. To determine the window of time for which dynamic MPF is guaranteed to produce observables with lower error than any of the individual Trotter formulas within the multi-product, we can implement the “MPF test” - see equation (17) and surrounding text in [\[3\]](#references).

#### Set up the Trotter circuits

At this point, we have found our expansion coefficients, $x$, and all that is left to do is to generate the Trotterized quantum circuits.
Once again, the [qiskit_addon_utils.problem_generators](/docs/api/qiskit-addon-utils/problem-generators) module comes to the rescue with a useful function to do this:

In [26]:
from qiskit.synthesis import SuzukiTrotter
from qiskit_addon_utils.problem_generators import (
    generate_time_evolution_circuit,
)
from qiskit import QuantumCircuit


total_time = 1.0
mpf_circuits = []
for k in mpf_trotter_steps:
    # Initial Neel state preparation
    circuit = QuantumCircuit(L)
    circuit.x([i for i in range(L) if i % 2 != 0])

    trotter_circ = generate_time_evolution_circuit(
        hamiltonian,
        synthesis=SuzukiTrotter(order=order, reps=k),
        time=total_time,
    )

    circuit.compose(trotter_circ, inplace=True)

    mpf_circuits.append(pm.run(circuit))

In [27]:
mpf_circuits[-1].draw("mpl", fold=-1, scale=0.4)

<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/87d2ac0c-0.avif" alt="Output of the previous code cell" />

### Step 2: Optimize problem for quantum hardware execution
Let's return to the calculation of the expectation value for a single time point. We'll pick a backend for executing the experiment on hardware.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService


service = QiskitRuntimeService()
backend = service.least_busy(min_num_qubits=127)
print(backend)

qubits = list(range(backend.num_qubits))

Notate che avete completa libertà su come risolvere questo problema di ottimizzazione, il che significa che potete cambiare il risolutore di ottimizzazione, le sue soglie di convergenza e così via.
Consultate la guida relativa su [Come utilizzare il modello approssimato.](https://qiskit.github.io/qiskit-addon-mpf/how_tos/using_approximate_model.html)
#### Coefficienti MPF dinamici
Nella sezione precedente, abbiamo introdotto una MPF statica che migliora l'approssimazione di Trotter standard. Tuttavia, questa versione statica non minimizza necessariamente l'errore di approssimazione. Concretamente, la MPF statica, indicata $\mu^S(t)$, non è la proiezione ottimale di $\rho(t)$ sul sottospazio spannato dagli stati della formula di prodotto ${\rho_{k_i}(t)}_{i=1}^r$.

Per affrontare questo, consideriamo una MPF dinamica (introdotta nel Rif. [\[2\]](#references) e dimostrata sperimentalmente nel Rif. [\[3\]](#references)) che minimizza l'errore di approssimazione nella norma di Frobenius. Formalmente, ci concentriamo sulla minimizzazione di

$$
\|\rho(t) - \mu^D(t)\|_F^2 \;=\; \mathrm{Tr}\bigl[ \left( \rho(t) - \mu^D(t)\right)^2 \bigr],
$$

rispetto ad alcuni coefficienti $x_i(t)$ ad ogni tempo $t$. Il proiettore *ottimale* nella norma di Frobenius è quindi $\mu^D(t) \;=\; \sum_{i=1}^r x_i(t)\,\rho_{k_i}(t)$, e chiamiamo $\mu^D(t)$ la MPF *dinamica*. Sostituendo le definizioni sopra:

$$
\|\rho(t) - \mu^D(t)\|_F^2
\;=\; \\
= \mathrm{Tr}\bigl[ \left( \rho(t) - \mu^D(t)\right)^2 \bigr]
\;=\; \\
= \mathrm{Tr}\bigl[ \left( \rho(t) - \sum_{i=1}^r x_i(t)\,\rho_{k_i}(t) \right) \left(  \rho(t) - \sum_{j=1}^r x_j(t)\,\rho_{k_j}(t) \right) \bigr]
\;=\; \\
= 1 \;+\; \sum_{i,j=1}^r M_{i,j}(t)\,x_i(t)\,x_j(t)
\;-\;
2 \sum_{i=1}^r L_i^{\mathrm{exact}}(t)\,x_i(t),
$$

dove $M_{i,j}(t)$ è la *matrice di Gram*, definita da

$$
M_{i,j}(t) \;=\; \mathrm{Tr}\bigl[\rho_{k_i}(t)\,\rho_{k_j}(t)\bigr]
\;=\;
\bigl|\langle \psi_{\mathrm{in}} \!\mid S\bigl(t/k_i\bigr)^{-k_i}\,S\bigl(t/k_j\bigr)^{k_j} \!\mid \psi_{\mathrm{in}} \rangle \bigr|^2.
$$

e

$$
L_i^{\mathrm{exact}}(t) = \mathrm{Tr}[\rho(t)\,\rho_{k_i}(t)]
$$

rappresenta la sovrapposizione tra lo stato esatto $\rho(t)$ e ciascuna approssimazione della formula di prodotto $\rho_{k_i}(t)$. In scenari pratici, queste sovrapposizioni possono essere misurate solo approssimativamente a causa del rumore o dell'accesso parziale a $\rho(t)$.

Qui, $\lvert\psi_{\mathrm{in}}\rangle$ è lo stato iniziale, e $S(\cdot)$ è l'operazione applicata nella formula di prodotto. Scegliendo i coefficienti $x_i(t)$ che minimizzano questa espressione (e gestendo i dati di sovrapposizione approssimati quando $\rho(t)$ non è completamente noto), otteniamo la "migliore" (in senso della norma di Frobenius) approssimazione dinamica di $\rho(t)$ all'interno del sottospazio MPF. Le quantità $L_i(t)$ e $M_{i,j}(t)$ possono essere calcolate in modo efficiente utilizzando metodi di reti tensoriali [\[3\]](#references). L'addon MPF di Qiskit fornisce diversi "backend" per eseguire il calcolo. L'esempio seguente mostra il modo più flessibile per farlo, e la documentazione del [backend basato su layer TeNPy](https://qiskit.github.io/qiskit-addon-mpf/apidocs/qiskit_addon_mpf.backends.tenpy_layers.html#module-qiskit_addon_mpf.backends.tenpy_layers) spiega anche in grande dettaglio. Per utilizzare questo metodo, partite dal circuito che implementa l'evoluzione temporale desiderata e create modelli che rappresentano queste operazioni dai layer del circuito corrispondente. Infine, viene creato un oggetto `Evolver` che può essere utilizzato per generare le quantità evolute nel tempo $M_{i,j}(t)$ e $L_i(t)$. Iniziamo creando l'oggetto `Evolver` corrispondente all'evoluzione temporale approssimata ([`ApproxEvolverFactory`](https://qiskit.github.io/qiskit-addon-mpf/apidocs/qiskit_addon_mpf.dynamic.html#qiskit_addon_mpf.dynamic.ApproxEvolverFactory)) implementata dai circuiti. In particolare, prestate particolare attenzione alla variabile `order` in modo che corrispondano. Notate che nel generare i circuiti corrispondenti all'evoluzione temporale approssimata, utilizziamo valori segnaposto per il `time = 1.0` e il numero di passi di Trotter (`reps=1`). I circuiti approssimanti corretti vengono quindi prodotti dal risolutore del problema dinamico in `setup_dynamic_lse`.

In [ ]:
import copy
from qiskit.transpiler import Target, CouplingMap

target = backend.target
instruction_2q = "cz"

cmap = target.build_coupling_map(filter_idle_qubits=True)
cmap_list = list(cmap.get_edges())

max_meas_err = 0.012
min_t2 = 40
max_twoq_err = 0.005

# Remove qubits with bad measurement or t2
cust_cmap_list = copy.deepcopy(cmap_list)
for q in range(target.num_qubits):
    meas_err = target["measure"][(q,)].error
    if target.qubit_properties[q].t2 is not None:
        t2 = target.qubit_properties[q].t2 * 1e6
    else:
        t2 = 0
    if meas_err > max_meas_err or t2 < min_t2:
        # print(q)
        for q_pair in cmap_list:
            if q in q_pair:
                try:
                    cust_cmap_list.remove(q_pair)
                except ValueError:
                    continue

# Remove qubits with bad 2q gate or t2
for q in cmap_list:
    twoq_gate_err = target[instruction_2q][q].error
    if twoq_gate_err > max_twoq_err:
        # print(q)
        for q_pair in cmap_list:
            if q == q_pair:
                try:
                    cust_cmap_list.remove(q_pair)
                except ValueError:
                    continue


cust_cmap = CouplingMap(cust_cmap_list)

cust_target = Target.from_configuration(
    basis_gates=backend.configuration().basis_gates
    + ["measure"],  # or whatever new set of gates
    coupling_map=cust_cmap,
)

sorted_components = sorted(
    [list(comp.physical_qubits) for comp in cust_cmap.connected_components()],
    reverse=True,
)
print("size of largest component", len(sorted_components[0]))

size of largest component 10


> **Warning:** Le opzioni di `LayerwiseEvolver` che determinano i dettagli della simulazione della rete tensoriale devono essere scelte con attenzione per evitare di impostare un problema di ottimizzazione mal definito.
Quindi impostiamo l'evolver esatto (ad esempio, [`ExactEvolverFactory`](https://qiskit.github.io/qiskit-addon-mpf/apidocs/qiskit_addon_mpf.dynamic.html#qiskit_addon_mpf.dynamic.ExactEvolverFactory)), che restituisce un oggetto [`Evolver`](https://qiskit.github.io/qiskit-addon-mpf/apidocs/qiskit_addon_mpf.backends.html#qiskit_addon_mpf.backends.Evolver) che calcola l'evoluzione temporale vera o "di riferimento". Realisticamente, approssimeremmo l'evoluzione esatta utilizzando una formula di Suzuki-Trotter di ordine superiore o un altro metodo affidabile con un piccolo passo temporale. Qui sotto, approssimiamo lo stato evoluto nel tempo esatto con una formula di Suzuki-Trotter del quarto ordine utilizzando un piccolo passo temporale `dt=0.1`, il che significa che il numero di passi di Trotter utilizzati al tempo $t$ è $k=t/dt$. Specifichiamo anche alcune opzioni di troncamento specifiche di TeNPy per limitare la dimensione massima del legame della rete tensoriale sottostante, così come i valori singolari minimi dei legami della rete tensoriale divisa. Questi parametri possono influenzare l'accuratezza del valore di aspettazione calcolato con i coefficienti MPF dinamici, quindi è importante esplorare un intervallo di valori per trovare l'equilibrio ottimale tra tempo di calcolo e accuratezza. Notate che il calcolo dei coefficienti MPF non si basa sul valore di aspettazione della PF ottenuto dall'esecuzione hardware, e pertanto può essere sintonizzato in post-elaborazione.

In [34]:
cust_cmap.draw()

<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/c5d8e90b-0.avif" alt="Output of the previous code cell" />

Successivamente, create lo stato iniziale del vostro sistema in un formato compatibile con TeNPy (ad esempio, un `MPS_neel_state`=$\vert 0101...01 \rangle$). Questo imposta la funzione d'onda a molti corpi che evolverete nel tempo $\lvert\psi_{\mathrm{in}}\rangle$ come un tensore.

In [72]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

transpiler = generate_preset_pass_manager(
    optimization_level=3, target=cust_target
)

transpiled_circuits = [transpiler.run(circ) for circ in mpf_circuits]

qubits_layouts = [
    [
        idx
        for idx, qb in circuit.layout.initial_layout.get_physical_bits().items()
        if qb._register.name != "ancilla"
    ]
    for circuit in transpiled_circuits
]

transpiled_circuits = []
for circuit, layout in zip(mpf_circuits, qubits_layouts):
    transpiler = generate_preset_pass_manager(
        optimization_level=3, backend=backend, initial_layout=layout
    )
    transpiled_circuit = transpiler.run(circuit)
    transpiled_circuits.append(transpiled_circuit)


# transform the observable defined on virtual qubits to
# an observable defined on all physical qubits
isa_observables = [
    observable.apply_layout(circ.layout) for circ in transpiled_circuits
]

In [73]:
print(transpiled_circuits[-1].depth(lambda x: x.operation.num_qubits == 2))
print(transpiled_circuits[-1].count_ops())
transpiled_circuits[-1].draw("mpl", idle_wires=False, fold=False)

51
OrderedDict([('sx', 310), ('rz', 232), ('cz', 132), ('x', 19)])


<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/25ce07a6-1.avif" alt="Output of the previous code cell" />

### Step 3: Execute using Qiskit primitives
With the Estimator primitive we can obtain the estimation of expectation value from the QPU. We execute the optimized AQC circuits with additional error mitigation and suppression techniques.

In [ ]:
from qiskit_ibm_runtime import EstimatorV2 as Estimator


estimator = Estimator(mode=backend)
estimator.options.default_shots = 30000

# Set simple error suppression/mitigation options
estimator.options.dynamical_decoupling.enable = True
estimator.options.twirling.enable_gates = True
estimator.options.twirling.enable_measure = True
estimator.options.twirling.num_randomizations = "auto"
estimator.options.twirling.strategy = "active-accum"
estimator.options.resilience.measure_mitigation = True
estimator.options.experimental.execution_path = "gen3-turbo"

estimator.options.resilience.zne_mitigation = True
estimator.options.resilience.zne.noise_factors = (1, 3, 5)
estimator.options.resilience.zne.extrapolator = ("exponential", "linear")

estimator.options.environment.job_tags = ["mpf small"]


job = estimator.run(
    [
        (circ, observable)
        for circ, observable in zip(transpiled_circuits, isa_observables)
    ]
)

Infine, tracciamo questi valori di aspettazione durante il tempo di evoluzione.

In [39]:
result_exp = job.result()
evs_exp = [res.data.evs for res in result_exp]
evs_std = [res.data.stds for res in result_exp]

print(evs_exp)

[array(-0.06361607), array(-0.23820448), array(-0.50271805)]


![Output of the previous code cell](../docs/images/tutorials/multi-product-formula/extracted-outputs/2da9c948-0.avif)

Nei casi come l'esempio sopra, dove la PF per $k=1$ si comporta male in tutti i tempi, anche la qualità dei risultati della MPF dinamica è fortemente influenzata. In tali situazioni, è utile indagare la possibilità di utilizzare singole PF con un numero maggiore di passi di Trotter per migliorare la qualità complessiva dei risultati. In queste simulazioni, vediamo l'interazione di diversi tipi di errori: errore dal campionamento finito ed errore di Trotter dalle formule di prodotto. La MPF aiuta a ridurre l'errore di Trotter dovuto alle formule di prodotto ma incorre in un errore di campionamento maggiore rispetto alle formule di prodotto. Questo può essere vantaggioso, poiché le formule di prodotto possono ridurre l'errore di campionamento con un campionamento maggiore, ma l'errore sistematico dovuto all'approssimazione di Trotter rimane intatto.

Un altro comportamento interessante che possiamo osservare dal grafico è che il valore di aspettazione per la PF per $k=1$ inizia a comportarsi in modo irregolare (oltre a non essere una buona approssimazione di quello esatto) ai tempi per i quali $t/k > 1 $, come spiegato nella [guida](https://qiskit.github.io/qiskit-addon-mpf/how_tos/choose_trotter_steps.html) su come scegliere il numero di passi di Trotter.

### Step 1: Mappare gli input classici a un problema quantistico
Consideriamo ora un singolo tempo $t=1.0$ e calcoliamo il valore di aspettazione della magnetizzazione con i vari metodi utilizzando una QPU. La scelta particolare di $t$ è stata fatta in modo da massimizzare la differenza tra i vari metodi e osservare la loro efficacia relativa. Per determinare la finestra temporale per cui l'MPF dinamico garantisce la produzione di osservabili con errore inferiore rispetto a qualsiasi delle singole formule di Trotter all'interno del multi-prodotto, possiamo implementare il "test MPF" - vedere l'equazione (17) e il testo circostante in [\[3\]](#references).
#### Configurare i circuiti di Trotter
A questo punto, abbiamo trovato i nostri coefficienti di espansione, $x$, e tutto ciò che resta da fare è generare i circuiti quantistici trotterizzati.
Ancora una volta, il modulo [qiskit_addon_utils.problem_generators](https://docs.quantum.ibm.com/api/qiskit-addon-utils/problem-generators) viene in soccorso con una funzione utile per farlo:

In [29]:
exact_mpf_std = np.sqrt(
    sum(
        [
            (coeff**2) * (std**2)
            for coeff, std in zip(coeffs_exact.value, evs_std)
        ]
    )
)
print(
    "Exact static MPF expectation value: ",
    evs_exp @ coeffs_exact.value,
    "+-",
    exact_mpf_std,
)
approx_mpf_std = np.sqrt(
    sum(
        [
            (coeff**2) * (std**2)
            for coeff, std in zip(coeffs_approx.value, evs_std)
        ]
    )
)
print(
    "Approximate static MPF expectation value: ",
    evs_exp @ coeffs_approx.value,
    "+-",
    approx_mpf_std,
)
dynamic_mpf_std = np.sqrt(
    sum(
        [
            (coeff**2) * (std**2)
            for coeff, std in zip(mpf_dynamic_coeffs_list[7], evs_std)
        ]
    )
)
print(
    "Dynamic MPF expectation value: ",
    evs_exp @ mpf_dynamic_coeffs_list[7],
    "+-",
    dynamic_mpf_std,
)

Exact static MPF expectation value:  -0.6329590442738475 +- 0.012798249760406036
Approximate static MPF expectation value:  -0.5690390035339492 +- 0.010459559917168473
Dynamic MPF expectation value:  -0.4655579758795695 +- 0.007639139186720507


Finally, for this small problem we can compute the exact reference value using [scipy.linalg.expm](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.expm.html) as follows:

In [30]:
from scipy.linalg import expm
from qiskit.quantum_info import Statevector

exp_H = expm(-1j * total_time * hamiltonian.to_matrix())

initial_state_circuit = QuantumCircuit(L)
initial_state_circuit.x([i for i in range(L) if i % 2 != 0])
initial_state = Statevector(initial_state_circuit).data

time_evolved_state = exp_H @ initial_state

exact_obs = (
    time_evolved_state.conj() @ observable.to_matrix() @ time_evolved_state
)
print("Exact expectation value ", exact_obs.real)

Exact expectation value  -0.39909900734489434


In [31]:
sym = {1: "^", 2: "s", 4: "p"}
# Get expectation values at all times for each Trotter step
for k, step in enumerate(mpf_trotter_steps):
    plt.errorbar(
        k,
        evs_exp[k],
        yerr=evs_std[k],
        alpha=0.5,
        markersize=4,
        marker=sym[step],
        color="grey",
        label=f"{mpf_trotter_steps[k]} Trotter steps",
    )  # , , )


plt.errorbar(
    3,
    evs_exp @ coeffs_exact.value,
    yerr=exact_mpf_std,
    markersize=4,
    marker="o",
    color="purple",
    label="Static MPF",
)

plt.errorbar(
    4,
    evs_exp @ coeffs_approx.value,
    yerr=approx_mpf_std,
    markersize=4,
    marker="o",
    color="orange",
    label="Approximate static MPF",
)

plt.errorbar(
    5,
    evs_exp @ mpf_dynamic_coeffs_list[7],
    yerr=dynamic_mpf_std,
    markersize=4,
    marker="o",
    color="pink",
    label="Dynamic MPF",
)

plt.axhline(
    y=exact_obs.real,
    linestyle="--",
    color="red",
    label="Exact time-evolution",
)


plt.title(
    f"Expectation values for (ZZ,{(L//2-1, L//2)}) at time {total_time} for the different methods "
)
plt.xlabel("Method")
plt.ylabel("Expectation Value")
plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.2), ncol=2)
plt.grid(alpha=0.1)
plt.tight_layout()
plt.show()

<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/a3eefe73-0.avif" alt="Output of the previous code cell" />

Quindi rimuoviamo i qubit anomali dalla mappa di accoppiamento per garantire che la fase di layout del transpiler non li includa. Di seguito utilizziamo le proprietà del backend riportate memorizzate nell'oggetto `target` e rimuoviamo i qubit che hanno errore di misura o porta a due qubit sopra una certa soglia (`max_meas_err`, `max_twoq_err`) oppure tempo $T_2$ (che determina la perdita di coerenza) al di sotto di una certa soglia (`min_t2`).

In [32]:
def relative_error(ev, exact_ev):
    return abs(ev - exact_ev)


relative_error_k = [relative_error(ev, exact_obs.real) for ev in evs_exp]
relative_error_mpf = relative_error(evs_exp @ mpf_coeffs, exact_obs.real)
relative_error_approx_mpf = relative_error(
    evs_exp @ coeffs_approx.value, exact_obs.real
)
relative_error_dynamic_mpf = relative_error(
    evs_exp @ mpf_dynamic_coeffs_list[7], exact_obs.real
)

print("relative error for each trotter steps", relative_error_k)
print("relative error with MPF exact coeffs", relative_error_mpf)
print("relative error with MPF approx coeffs", relative_error_approx_mpf)
print("relative error with MPF dynamic coeffs", relative_error_dynamic_mpf)

relative error for each trotter steps [0.33548293650112293, 0.16089452939226306, 0.10361904247828346]
relative error with MPF exact coeffs 0.2338600369291003
relative error with MPF approx coeffs 0.16993999618905486
relative error with MPF dynamic coeffs 0.06645896853467514


## Part II: scale it up

Let's scale the problem up beyond what is possible to simulate exactly. In this section we will focus on reproducing some of the results shown in Ref. [\[3\]](#references).

### Step 1: Map classical inputs to a quantum problem

#### Hamiltonian

For the large-scale example, we use the XXZ model on a line of 50 sites:

$$
\hat{\mathcal{H}}_{XXZ} = \sum_{i=1}^{L-1} J_{i,(i+1)}\left(X_i X_{(i+1)}+Y_i Y_{(i+1)}+ 2\cdot Z_i Z_{(i+1)} \right) \, ,
$$

where $J_{i,(i+1)}$ is a random coefficient corresponding to edge $(i, i+1)$. This is the Hamiltonian considered in the demonstration presented in Ref. [\[3\]](#references).

In [33]:
L = 50
# Generate some coupling map to use for this example
coupling_map = CouplingMap.from_line(L, bidirectional=False)
graphviz_draw(coupling_map.graph, method="circo")

<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/34bf68ac-0.avif" alt="Output of the previous code cell" />

In [ ]:
import numpy as np
from qiskit.quantum_info import SparsePauliOp, Pauli


# Generate random coefficients for our XXZ Hamiltonian
np.random.seed(0)
even_edges = list(coupling_map.get_edges())[::2]
odd_edges = list(coupling_map.get_edges())[1::2]

Js = np.random.uniform(0.5, 1.5, size=L)
hamiltonian = SparsePauliOp(Pauli("I" * L))
for i, edge in enumerate(even_edges + odd_edges):
    hamiltonian += SparsePauliOp.from_sparse_list(
        [
            ("XX", (edge), 2 * Js[i]),
            ("YY", (edge), 2 * Js[i]),
            ("ZZ", (edge), 4 * Js[i]),
        ],
        num_qubits=L,
    )

print(hamiltonian)

SparsePauliOp(['IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXX', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYY', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZ', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIIIIIIIII', 'IIIIIIIIIIII

![Output of the previous code cell](../docs/images/tutorials/multi-product-formula/extracted-outputs/c5d8e90b-0.avif)

Possiamo quindi mappare il circuito e l'osservabile sui qubit fisici del dispositivo.

In [35]:
observable = SparsePauliOp.from_sparse_list(
    [("ZZ", (L // 2 - 1, L // 2), 1.0)], num_qubits=L
)
print(observable)

SparsePauliOp(['IIIIIIIIIIIIIIIIIIIIIIIIZZIIIIIIIIIIIIIIIIIIIIIIII'],
              coeffs=[1.+0.j])


#### Choose Trotter steps
The experiment showcased in Fig. 4 of Ref. [\[3\]](#references) uses $k_j = [2, 3, 4]$ symmetric Trotter steps of order $2$. We focus on the results for time $t=3$, where the MPF and a PF with a higher number of Trotter steps (6 in this case) have the same Trotter error. However, the MPF expectation value is calculated from circuits corresponding to the lower number of Trotter steps and thus shallower. In practice, even if the MPF and the deeper Trotter steps circuit have the same Trotter error, we expect the experimental expectation value calculated from the MPF circuits to be closer to the theory one, as it entails running shallower circuits less exposed to hardware noise compared to the circuit corresponding to the higher Trotter step PF.

In [36]:
total_time = 3
mpf_trotter_steps = [2, 3, 4]
order = 2
symmetric = True

#### Set up the LSE
Here we look at the static MPF coefficients for this problem.

In [37]:
lse = setup_static_lse(mpf_trotter_steps, order=order, symmetric=symmetric)
mpf_coeffs = lse.solve()
print(
    f"The static coefficients associated with the ansatze are: {mpf_coeffs}"
)
print("L1 norm:", np.linalg.norm(mpf_coeffs, ord=1))

The static coefficients associated with the ansatze are: [ 0.26666667 -2.31428571  3.04761905]
L1 norm: 5.628571428571431


In [38]:
model_approx, coeffs_approx = setup_sum_of_squares_problem(
    lse, max_l1_norm=2.0
)
model_approx.solve()
print(coeffs_approx.value)
print(
    "L1 norm of the approximate coefficients:",
    np.linalg.norm(coeffs_approx.value, ord=1),
)

[-0.24255546 -0.25744454  1.5       ]
L1 norm of the approximate coefficients: 2.0


### Step 4: Post-elaborare e restituire il risultato nel formato classico desiderato
L'unico passo di post-elaborazione consiste nel combinare il valore di aspettazione ottenuto dalle primitive di Qiskit Runtime a diversi passi di Trotter utilizzando i rispettivi coefficienti MPF. Per un osservabile $A$ abbiamo:

$$ \langle A \rangle_{\text{mpf}}  = \text{Tr} [A \mu(t)] = \sum_{j} x_j  \text{Tr} [A \rho_{k_j}] = \sum_{j} x_j \langle A \rangle_j$$

Prima di tutto, estraiamo i singoli valori di aspettazione ottenuti per ciascuno dei circuiti di Trotter:

In [ ]:
# Create approximate time-evolution circuits
single_2nd_order_circ = generate_time_evolution_circuit(
    hamiltonian, time=1.0, synthesis=SuzukiTrotter(reps=1, order=order)
)
single_2nd_order_circ = pm.run(single_2nd_order_circ)  # collect XX and YY

# Find layers in the circuit
layers = slice_by_depth(single_2nd_order_circ, max_slice_depth=1)

# Create tensor network models
models = [
    LayerModel.from_quantum_circuit(layer, conserve="Sz") for layer in layers
]

# Create the time-evolution object
approx_factory = partial(
    LayerwiseEvolver,
    layers=models,
    options={
        "preserve_norm": False,
        "trunc_params": {
            "chi_max": 64,
            "svd_min": 1e-8,
            "trunc_cut": None,
        },
        "max_delta_t": 4,
    },
)

# Create exact time-evolution circuits
single_4th_order_circ = generate_time_evolution_circuit(
    hamiltonian, time=1.0, synthesis=SuzukiTrotter(reps=1, order=4)
)
single_4th_order_circ = pm.run(single_4th_order_circ)
exact_model_layers = [
    LayerModel.from_quantum_circuit(layer, conserve="Sz")
    for layer in slice_by_depth(single_4th_order_circ, max_slice_depth=1)
]

# Create the time-evolution object
exact_factory = partial(
    LayerwiseEvolver,
    layers=exact_model_layers,
    dt=0.1,
    options={
        "preserve_norm": False,
        "trunc_params": {
            "chi_max": 64,
            "svd_min": 1e-8,
            "trunc_cut": None,
        },
        "max_delta_t": 3,
    },
)


def identity_factory():
    return MPOState.initialize_from_lattice(models[0].lat, conserve=True)


mps_initial_state = MPS_neel_state(models[0].lat)


lse = setup_dynamic_lse(
    mpf_trotter_steps,
    total_time,
    identity_factory,
    exact_factory,
    approx_factory,
    mps_initial_state,
)
problem, coeffs = setup_frobenius_problem(lse)
try:
    problem.solve()
    mpf_dynamic_coeffs = coeffs.value
except Exception as error:
    print(error, "Calculation Failed for time", total_time)
print("")

#### Construct each of the Trotter circuits in our MPF decomposition

In [41]:
from qiskit.synthesis import SuzukiTrotter
from qiskit_addon_utils.problem_generators import (
    generate_time_evolution_circuit,
)
from qiskit import QuantumCircuit


mpf_circuits = []
for k in mpf_trotter_steps:
    # Initial state preparation |1010..>
    circuit = QuantumCircuit(L)
    circuit.x([i for i in range(L) if i % 2])

    trotter_circ = generate_time_evolution_circuit(
        hamiltonian,
        synthesis=SuzukiTrotter(reps=k, order=order),
        time=total_time,
    )

    circuit.compose(trotter_circ, qubits=range(L), inplace=True)

    mpf_circuits.append(circuit)

Successivamente, li ricombiniamo semplicemente con i nostri coefficienti MPF per ottenere i valori di aspettazione totali dell'MPF. Di seguito, lo facciamo per ciascuno dei diversi modi con cui abbiamo calcolato $x$.

In [42]:
k = 6

# Initial state preparation |1010..>
comp_circuit = QuantumCircuit(L)
comp_circuit.x([i for i in range(L) if i % 2])


trotter_circ = generate_time_evolution_circuit(
    hamiltonian,
    synthesis=SuzukiTrotter(reps=k, order=order),
    time=total_time,
)

comp_circuit.compose(trotter_circ, qubits=range(L), inplace=True)


mpf_circuits.append(comp_circuit)

### Step 2: Optimize problem for quantum hardware execution

In [ ]:
import copy
from qiskit.transpiler import Target, CouplingMap

target = backend.target
instruction_2q = "cz"

cmap = target.build_coupling_map(filter_idle_qubits=True)
cmap_list = list(cmap.get_edges())

max_meas_err = 0.055
min_t2 = 30
max_twoq_err = 0.01

# Remove qubits with bad measurement or t2
cust_cmap_list = copy.deepcopy(cmap_list)
for q in range(target.num_qubits):
    meas_err = target["measure"][(q,)].error
    if target.qubit_properties[q].t2 is not None:
        t2 = target.qubit_properties[q].t2 * 1e6
    else:
        t2 = 0
    if meas_err > max_meas_err or t2 < min_t2:
        # print(q)
        for q_pair in cmap_list:
            if q in q_pair:
                try:
                    cust_cmap_list.remove(q_pair)
                except ValueError:
                    continue

# Remove qubits with bad 2q gate or t2
for q in cmap_list:
    twoq_gate_err = target[instruction_2q][q].error
    if twoq_gate_err > max_twoq_err:
        # print(q)
        for q_pair in cmap_list:
            if q == q_pair:
                try:
                    cust_cmap_list.remove(q_pair)
                except ValueError:
                    continue


cust_cmap = CouplingMap(cust_cmap_list)

cust_target = Target.from_configuration(
    basis_gates=backend.configuration().basis_gates
    + ["measure"],  # or whatever new set of gates
    coupling_map=cust_cmap,
)

sorted_components = sorted(
    [list(comp.physical_qubits) for comp in cust_cmap.connected_components()],
    reverse=True,
)
print("size of largest component", len(sorted_components[0]))

size of largest component 73


In [104]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

transpiler = generate_preset_pass_manager(
    optimization_level=3, target=cust_target
)

transpiled_circuits = [transpiler.run(circ) for circ in mpf_circuits]

qubits_layouts = [
    [
        idx
        for idx, qb in circuit.layout.initial_layout.get_physical_bits().items()
        if qb._register.name != "ancilla"
    ]
    for circuit in transpiled_circuits
]

transpiled_circuits = []
for circuit, layout in zip(mpf_circuits, qubits_layouts):
    transpiler = generate_preset_pass_manager(
        optimization_level=3, backend=backend, initial_layout=layout
    )
    transpiled_circuit = transpiler.run(circuit)
    transpiled_circuits.append(transpiled_circuit)


# transform the observable defined on virtual qubits to
# an observable defined on all physical qubits
isa_observables = [
    observable.apply_layout(circ.layout) for circ in transpiled_circuits
]

### Step 3: Execute using Qiskit primitives

In [ ]:
from qiskit_ibm_runtime import EstimatorV2 as Estimator

estimator = Estimator(mode=backend)
estimator.options.default_shots = 30000

# Set simple error suppression/mitigation options
estimator.options.dynamical_decoupling.enable = True
estimator.options.twirling.enable_gates = True
estimator.options.twirling.enable_measure = True
estimator.options.twirling.num_randomizations = "auto"
estimator.options.twirling.strategy = "active-accum"
estimator.options.resilience.measure_mitigation = True
estimator.options.experimental.execution_path = "gen3-turbo"

estimator.options.resilience.zne_mitigation = True
estimator.options.resilience.zne.noise_factors = (1, 1.2, 1.4)
estimator.options.resilience.zne.extrapolator = "linear"

estimator.options.environment.job_tags = ["mpf large"]

job_50 = estimator.run(
    [
        (circ, observable)
        for circ, observable in zip(transpiled_circuits, isa_observables)
    ]
)

### Step 4: Post-process and return result in desired classical format

In [46]:
result = job_50.result()
evs = [res.data.evs for res in result]
std = [res.data.stds for res in result]

print(evs)
print(std)

[array(-0.08034071), array(-0.00605026), array(-0.15345759), array(-0.18127293)]
[array(0.04482517), array(0.03438413), array(0.21540776), array(0.21520829)]


In [52]:
exact_mpf_std = np.sqrt(
    sum([(coeff**2) * (std**2) for coeff, std in zip(mpf_coeffs, std[:3])])
)
print(
    "Exact static MPF expectation value: ",
    evs[:3] @ mpf_coeffs,
    "+-",
    exact_mpf_std,
)
approx_mpf_std = np.sqrt(
    sum(
        [
            (coeff**2) * (std**2)
            for coeff, std in zip(coeffs_approx.value, std[:3])
        ]
    )
)
print(
    "Approximate static MPF expectation value: ",
    evs[:3] @ coeffs_approx.value,
    "+-",
    approx_mpf_std,
)
dynamic_mpf_std = np.sqrt(
    sum(
        [
            (coeff**2) * (std**2)
            for coeff, std in zip(mpf_dynamic_coeffs, std[:3])
        ]
    )
)
print(
    "Dynamic MPF expectation value: ",
    evs[:3] @ mpf_dynamic_coeffs,
    "+-",
    dynamic_mpf_std,
)

Exact static MPF expectation value:  -0.47510243192011536 +- 0.6613940032465087
Approximate static MPF expectation value:  -0.20914170384216998 +- 0.32341567460419135
Dynamic MPF expectation value:  -0.07994951978722761 +- 0.07423091963310202


In [ ]:
sym = {2: "^", 3: "s", 4: "p"}
# Get expectation values at all times for each Trotter step
for k, step in enumerate(mpf_trotter_steps):
    plt.errorbar(
        k,
        evs[k],
        yerr=std[k],
        alpha=0.5,
        markersize=4,
        marker=sym[step],
        color="grey",
        label=f"{mpf_trotter_steps[k]} Trotter steps",
    )


plt.errorbar(
    3,
    evs[-1],
    yerr=std[-1],
    alpha=0.5,
    markersize=8,
    marker="x",
    color="blue",
    label="6 Trotter steps",
)


plt.errorbar(
    4,
    evs[:3] @ mpf_coeffs,
    yerr=exact_mpf_std,
    markersize=4,
    marker="o",
    color="purple",
    label="Static MPF",
)

plt.errorbar(
    5,
    evs[:3] @ coeffs_approx.value,
    yerr=approx_mpf_std,
    markersize=4,
    marker="o",
    color="orange",
    label="Approximate static MPF",
)

plt.errorbar(
    6,
    evs[:3] @ mpf_dynamic_coeffs,
    yerr=dynamic_mpf_std,
    markersize=4,
    marker="o",
    color="pink",
    label="Dynamic MPF",
)

exact_obs = -0.24384471447172074  # Calculated via Tensor Network calculation
plt.axhline(
    y=exact_obs, linestyle="--", color="red", label="Exact time-evolution"
)


plt.title(
    f"Expectation values for (ZZ,{(L//2-1, L//2)}) at time {total_time} for the different methods "
)
plt.xlabel("Method")
plt.ylabel("Expectation Value")
plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.2), ncol=2)
plt.grid(alpha=0.1)
plt.tight_layout()
plt.show()

<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/d751af7c-0.avif" alt="Output of the previous code cell" />

## Parte II: aumentare la scala
Aumentiamo la scala del problema oltre ciò che è possibile simulare esattamente. In questa sezione ci concentreremo sulla riproduzione di alcuni dei risultati mostrati nel Rif. [\[3\]](#references).
### Passo 1: Mappare input classici a un problema quantistico
#### Hamiltoniana
Per l'esempio su larga scala, utilizziamo il modello XXZ su una linea di 50 siti:

$$
\hat{\mathcal{H}}_{XXZ} = \sum_{i=1}^{L-1} J_{i,(i+1)}\left(X_i X_{(i+1)}+Y_i Y_{(i+1)}+ 2\cdot Z_i Z_{(i+1)} \right) \, ,
$$

dove $J_{i,(i+1)}$ è un coefficiente casuale corrispondente allo spigolo $(i, i+1)$. Questa è l'Hamiltoniana considerata nella dimostrazione presentata nel Rif. [\[3\]](#references).